# RX 9070 XT DirectML Testing

Moving from my README.md notes to this notebook for faster iteration. I can get PyTorch to work on DirectML but it feels hacky and I need to work out the details of what exact packages I need etc... I would like to get the requirements locked down so I can test (even if very slow compared to native/ROCm) on this new GPU.

#### pip list - Current semi-working state

```
Package                   Version
------------------------- ---------------
aiofiles                  23.2.1
altair                    5.5.0
annotated-types           0.7.0
anyio                     4.9.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.3
bleach                    6.2.0
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.1
cycler                    0.12.1
debugpy                   1.8.13
decorator                 5.2.1
defusedxml                0.7.1
diffusers                 0.32.2
executing                 2.2.0
fastapi                   0.115.11
fastjsonschema            2.21.1
ffmpy                     0.5.0
filelock                  3.18.0
fonttools                 4.56.0
fqdn                      1.5.1
fsspec                    2025.3.0
gradio                    3.48.0
gradio_client             0.6.1
groovy                    0.1.2
h11                       0.14.0
httpcore                  1.0.7
httpx                     0.28.1
huggingface-hub           0.29.3
idna                      3.10
importlib_metadata        8.6.1
importlib_resources       6.5.2
ipykernel                 6.29.5
ipython                   9.0.2
ipython_pygments_lexers   1.1.1
isoduration               20.11.0
jedi                      0.19.2
Jinja2                    3.1.6
json5                     0.10.0
jsonpointer               3.0.0
jsonschema                4.23.0
jsonschema-specifications 2024.10.1
jupyter_client            8.6.3
jupyter_core              5.7.2
jupyter-events            0.12.0
jupyter-lsp               2.2.5
jupyter_server            2.15.0
jupyter_server_terminals  0.5.3
jupyterlab                4.3.6
jupyterlab_pygments       0.3.0
jupyterlab_server         2.27.3
kiwisolver                1.4.8
markdown-it-py            3.0.0
MarkupSafe                2.1.5
matplotlib                3.10.1
matplotlib-inline         0.1.7
mdurl                     0.1.2
mistune                   3.1.3
mpmath                    1.3.0
narwhals                  1.31.0
nbclient                  0.10.2
nbconvert                 7.16.6
nbformat                  5.10.4
nest-asyncio              1.6.0
networkx                  3.4.2
notebook_shim             0.2.4
numpy                     1.26.4
orjson                    3.10.15
overrides                 7.7.0
packaging                 24.2
pandas                    2.2.3
pandocfilters             1.5.1
parso                     0.8.4
pillow                    10.4.0
pip                       25.0
platformdirs              4.3.6
prometheus_client         0.21.1
prompt_toolkit            3.0.50
psutil                    7.0.0
pure_eval                 0.2.3
pycparser                 2.22
pydantic                  2.10.6
pydantic_core             2.27.2
pydub                     0.25.1
Pygments                  2.19.1
pyparsing                 3.2.1
python-dateutil           2.9.0.post0
python-json-logger        3.3.0
python-multipart          0.0.20
pytz                      2025.1
pywin32                   310
pywinpty                  2.0.15
PyYAML                    6.0.2
pyzmq                     26.3.0
referencing               0.36.2
regex                     2024.11.6
requests                  2.32.3
rfc3339-validator         0.1.4
rfc3986-validator         0.1.1
rich                      13.9.4
rpds-py                   0.23.1
ruff                      0.11.0
safehttpx                 0.1.6
safetensors               0.5.3
scipy                     1.15.2
semantic-version          2.10.0
Send2Trash                1.8.3
setuptools                75.8.0
shellingham               1.5.4
six                       1.17.0
sniffio                   1.3.1
soupsieve                 2.6
stack-data                0.6.3
starlette                 0.46.1
sympy                     1.13.3
terminado                 0.18.1
tinycss2                  1.4.0
tokenizers                0.21.1
tomlkit                   0.13.2
torch                     2.4.1
torch-directml            0.2.4.dev240913
torchvision               0.19.1
tornado                   6.4.2
tqdm                      4.67.1
traitlets                 5.14.3
transformers              4.49.0
typer                     0.15.2
types-python-dateutil     2.9.0.20241206
typing_extensions         4.12.2
tzdata                    2025.1
uri-template              1.3.0
urllib3                   2.3.0
uvicorn                   0.34.0
wcwidth                   0.2.13
webcolors                 24.11.1
webencodings              0.5.1
websocket-client          1.8.0
websockets                11.0.3
wheel                     0.45.1
zipp                      3.21.0
```

#### DirectML Basic Test

In [ ]:
import torch
import torch_directml
dml = torch_directml.device()

tensor1 = torch.tensor([5]).to(dml)
tensor2 = torch.tensor([5]).to(dml)

dml_algebra = tensor1 * tensor2
dml_algebra.item()

print(dml_algebra)

#### JSON Bert Test Setup

In [ ]:
import os
import json
import random

# Constants
TRAIN_GOOD_FILE_COUNT = 250
TEST_GOOD_FILE_COUNT = 50
TRAIN_BAD_FILE_COUNT = 250
TEST_BAD_FILE_COUNT = 50
DATA_FOLDER = "data"
WDSDataset_FOLDER = os.path.join(DATA_FOLDER, "benchmark_dataset")

# Generate Good files
good_files = []
for i in range(TRAIN_GOOD_FILE_COUNT + TEST_GOOD_FILE_COUNT):
    data = {
        "id": random.randint(100000, 999999),
        "name": {
            "first": f"John",
            "last": f"Doe {i+1}"
        },
        "age": random.randint(18, 60),
        "score": {
            "math": round(random.uniform(0.6, 1.0), 2),
            "science": round(random.uniform(0.6, 1.0), 2),
            "english": round(random.uniform(0.6, 1.0), 2)
        },
        "address": {
            "street": f"{random.randint(100, 999)} Main St",
            "city": "Anytown",
            "state": "CA",
            "zip": f"{random.randint(10000, 99999)}"
        },
        "contacts": [
            {"type": "email", "value": f"john.doe{i+1}@example.com"},
            {"type": "phone", "value": f"555-{random.randint(1000, 9999)}"}
        ]
    }
    good_files.append(json.dumps(data))

# Generate Bad files
bad_files = []
for i in range(TRAIN_BAD_FILE_COUNT + TEST_BAD_FILE_COUNT):
    data = {
        "id": random.randint(100000, 999999),
        "name": {
            "first": f"John",
            "last": f"Doe {i+1}"
        },
        "age": random.randint(18, 60),
        "score": {
            "math": round(random.uniform(0.4, 0.9), 2),
            "science": round(random.uniform(0.4, 0.9), 2),
            "english": round(random.uniform(0.4, 0.9), 2)
        },
        "address": {
            "street": f"{random.randint(100, 999)} Main St",
            "city": "Anytown",
            "state": "CA",
            "zip": f"{random.randint(10000, 99999)}"
        },
        "contacts": [
            {"type": "email", "value": f"john.doe{i+1}@example.com"},
            {"type": "phone", "value": f"555-{random.randint(1000, 9999)}"}
        ],
        "feedback": ["bad", "not good", "block"][random.randint(0, 2)]
    }
    bad_files.append(json.dumps(data))

# Create train and test folder structure
train_good_folder_path = os.path.join(WDSDataset_FOLDER, "train", "good")
train_bad_folder_path = os.path.join(WDSDataset_FOLDER, "train", "bad")
test_good_folder_path = os.path.join(WDSDataset_FOLDER, "test", "good")
test_bad_folder_path = os.path.join(WDSDataset_FOLDER, "test", "bad")

os.makedirs(train_good_folder_path, exist_ok=True)
os.makedirs(train_bad_folder_path, exist_ok=True)
os.makedirs(test_good_folder_path, exist_ok=True)
os.makedirs(test_bad_folder_path, exist_ok=True)

# Save Good files to train/GOOD and test/GOOD folders
for i in range(TRAIN_GOOD_FILE_COUNT):
    good_file_path = os.path.join(train_good_folder_path, f"good{i+1}.json")
    with open(good_file_path, "w") as f:
        f.write(good_files[i])

for i in range(TEST_GOOD_FILE_COUNT):
    good_file_path = os.path.join(train_good_folder_path, f"good{i+1}.json")
    with open(good_file_path, "w") as f:
        f.write(good_files[TRAIN_GOOD_FILE_COUNT + i])

# Save Bad files to train/BAD and test/BAD folders
for i in range(TRAIN_BAD_FILE_COUNT):
    bad_file_path = os.path.join(train_bad_folder_path, f"bad{i+1}.json")
    with open(bad_file_path, "w") as f:
        f.write(bad_files[i])

for i in range(TEST_BAD_FILE_COUNT):
    bad_file_path = os.path.join(train_bad_folder_path, f"bad{i+1}.json")
    with open(bad_file_path, "w") as f:
        f.write(bad_files[TRAIN_BAD_FILE_COUNT + i])

In [ ]:
import os
os.environ["DISABLE_TORCH_HALF"] = "1"

import torch
import torch_directml
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import time
import tqdm

# Set device
device = torch_directml.device(0)
print(f"Device: {device}")

# Synthetic Dataset
class SyntheticDataset(Dataset):
    def __init__(self, num_samples=1000, img_size=32, num_classes=2):
        self.num_samples = num_samples
        self.data = torch.randn(num_samples, 3, img_size, img_size)  # [N, C, H, W]
        self.labels = torch.randint(0, num_classes, (num_samples,))  # Binary labels
        print(f"Generated {num_samples} synthetic samples.")

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return {
            'input': self.data[idx],
            'label': self.labels[idx]
        }

# Simple CNN Model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)  # 3 input channels -> 16
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)  # 16 -> 32
        self.pool = nn.MaxPool2d(2, 2)  # Downsample by 2x
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # After 2 pooling layers: 32x8x8
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))  # [B, 16, 16, 16]
        x = self.pool(self.relu(self.conv2(x)))  # [B, 32, 8, 8]
        x = x.view(x.size(0), -1)  # Flatten: [B, 32*8*8]
        x = self.relu(self.fc1(x))  # [B, 128]
        x = self.fc2(x)  # [B, num_classes]
        return x

# Initialize model, loss, and optimizer
model = SimpleCNN(num_classes=2).to(device).float()
print(f"Model device: {next(model.parameters()).device}")
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Data
train_dataset = SyntheticDataset(num_samples=1000)
test_dataset = SyntheticDataset(num_samples=200)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

# Training function
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    print("Starting training epoch...")
    epoch_start = time.time()
    for step, batch in enumerate(train_loader):
        data_start = time.time()
        inputs = batch['input'].to(device)
        labels = batch['label'].to(device)
        data_time = time.time() - data_start
        step_start = time.time()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        step_time = time.time() - step_start
        print(f"Step {step}: Loss = {loss.item():.4f}, Data Time = {data_time:.3f}s, Compute Time = {step_time:.3f}s")
    
    epoch_time = time.time() - epoch_start
    print(f"Training epoch complete. Total time = {epoch_time:.3f}s")
    return total_loss / len(train_loader)

# Testing function
def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    print("Starting testing...")
    with torch.no_grad():
        for batch in test_loader:
            inputs = batch['input'].to(device)
            labels = batch['label'].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print("Testing complete.")
    return total_loss / len(test_loader), accuracy

# Training loop
epochs = 5
start_time = time.time()
for epoch in tqdm.tqdm(range(epochs)):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    test_loss, accuracy = test_model(model, test_loader, criterion)
    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")

In [ ]:
import torch_directml
print(torch_directml.device_count())  # Should print > 0
print(torch_directml.device_name(0))  # Should show your AMD GPU